In [1]:
import numpy as np
import pandas as pd
import unidecode

# Criando o DataFrame geral
### Carregando os dados principais:

In [2]:
casos_df = pd.read_csv('datasets/dados_casos.csv')
df_pib = pd.read_excel('datasets/PIB_2010-2019.xls')
vax_df = pd.read_csv('datasets/dados_vacinacao.csv')
dem_df = pd.read_csv('datasets/den_demografica-PB2010.csv')

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

### Começando a criação do general_df

Uma vez que a maioria dos dados utilizados são do DataFrame de casos, inicialzaremos a variávl general_df recebendo o DataFrame casos_df.

In [ ]:
general_df = casos_df

Para evitar erros, as strings correspondentes às cidades serão convertidas para Unidecode, sem acentos ou cedilhas, e maiúsculas. Essa correção será feita em todos os DataFrames.

In [ ]:
general_df.city = general_df.city.apply(unidecode.unidecode).str.upper().str.strip()
general_df = general_df.drop(columns= ['date', 'state', 'place_type', 'is_last', 'city_ibge_code'])
general_df.head(5)

## Adicionando o PIB ao DataFrame:

Dados sobre o PIB de todo o país:

In [ ]:
df_pib

Tendo em vista que as análises do relatório serão restritas à Paraíba e há maior interesse em dados mais novos, a variável pb_pib_df será criada recebendo apenas esses dados. Além disso, a variável será restrita ao nome do município e o seu ib per capita associado.

In [ ]:
pb_pib_df = df_pib[(df_pib['Sigla da Unidade da Federação'] == 'PB') & (df_pib['Ano'] == 2019)]

pb_pib_df = pb_pib_df.loc[:, ['Nome do Município', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]

pb_pib_df.columns = ['city', 'pib_per_capita']

pb_pib_df.reset_index(drop = True, inplace = True)

In [ ]:
pb_pib_df.city = pb_pib_df.city.apply(unidecode.unidecode).str.upper().str.strip()
pb_pib_df

In [ ]:
general_df = general_df.set_index('city').join(pb_pib_df.set_index('city'))
general_df.head(2)

## Adicionando o número de vacinados ao DataFrame:

Para descobrir o número de vacinados por mnicípio, será feito um value_counts em cima do estabelecimento que vacinou cada município.

In [ ]:
vax_df.head(3)

In [ ]:
count_df = pd.DataFrame(vax_df['estabelecimento_municipio_nome'].value_counts()).sort_index()
count_df.reset_index(inplace= True)
count_df.columns = ['city', 'n_vaccinations']
count_df

In [ ]:
general_df = general_df.join(count_df.set_index('city'))

In [ ]:
general_df

## Adicionando a densidade demográfica ao DataFrame:

Para adicionar a densidade demográfica, bastou padrozinar o gráfico e juntar ao DataFrame principal.

In [ ]:
dem_df.cidade = dem_df.cidade.apply(unidecode.unidecode).str.upper().str.strip()

In [ ]:
dem_df.columns = ['city', 'demographic_density']
dem_df

In [ ]:
general_df =general_df.join(dem_df.set_index('city'))

In [ ]:
general_df.reset_index(inplace= True)

In [ ]:
general_df.head(2)

## Salvando o DataSet:

Feitas as alterações, adicionamos os dados a um csv na pasta de DataSets:

In [ ]:
general_df.to_csv('datasets/dados_pb.csv', index= False)